# Minimal Usage Example

This provides an minimal usage example for `pydantic-cereal`.
To start, use the following imports and create a global `cereal` object:

In [1]:
"""Minimal imports."""

from typing import NewType
from upath import UPath
from pydantic import BaseModel, ConfigDict
from pydantic_cereal import Cereal

cereal = Cereal()  # global variable

We have a custom type `MyType`, in this case it's just an alias for `str`:

In [2]:
MyType = NewType("MyType", str)  # actually `str`, but type checker has special semantics 

We must add reader and writer classes for it. 
These must accept [`fsspec`](https://filesystem-spec.readthedocs.io/en/latest/) URIs as inputs.
We can register these with our `cereal` object by creating a wrapped (`Annotated`) type.

In [3]:
def my_reader(uri: str) -> MyType:
    """Read the object from an fsspec URI."""
    return MyType(UPath(uri).read_text())

def my_writer(obj: MyType, uri: str) -> None:
    """Write the object to an fsspec URI."""
    UPath(uri).write_text(obj)

MyWrappedType = cereal.wrap_type(MyType, reader=my_reader, writer=my_writer)

Note that your type checker should recognize `MyWrappedType` as exactly `MyType`.

In [4]:
from typing import get_args  # noqa

assert get_args(MyWrappedType)[0] == MyType

We can use this type in a Pydantic model:

In [5]:
class ExampleModel(BaseModel):
    """Example model."""

    model_config = ConfigDict(arbitrary_types_allowed=True)

    fld: MyWrappedType  # NOTE: Make sure to use the wrapped type!
    value: str = "default_value"

You can instantiate objects as usual:

In [6]:
mdl = ExampleModel(fld=MyType("my_field"))

Now, you can write your model to an arbitrary directory-like `fsspec` URI.
In this example, we're writing to a temporary [`MemoryFileSystem`](https://filesystem-spec.readthedocs.io/en/latest/api.html#fsspec.implementations.memory.MemoryFileSystem):

In [7]:
cereal.write_model(mdl, "memory://example_model")

MemoryPath('memory://example_model/')

And we can load another object from there:

In [8]:
obj = cereal.read_model("memory://example_model")
assert isinstance(obj, ExampleModel)
obj.fld

'my_field'

If you require a specific type (or base type), you can specify this in `read_model`:

In [9]:
cereal.read_model("memory://example_model", supercls=ExampleModel).fld

'my_field'

Inspecting the path, you can see the file structure:

In [10]:
list(UPath("memory://example_model").glob("*"))

[MemoryPath('memory://example_model/51c07fc879fa403993ba780d9ff29b52'),
 MemoryPath('memory://example_model/model.json'),
 MemoryPath('memory://example_model/model.schema.json')]